<a href="https://colab.research.google.com/github/RDelg/rl-book/blob/master/chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import random

In [4]:
class BlackJack():
    
    _CARDS = {
        '_1': 1,
        '_2': 2,
        '_3': 3,
        '_4': 4,
        '_5': 5,
        '_6': 6,
        '_7': 7,
        '_8': 8,
        '_9': 9,
        '_J': 10,
        '_Q': 10,
        '_K': 10,
        '_A': 1
    }

    @staticmethod
    def get_card():
        return random.choice(list(BlackJack._CARDS.keys()))
    
    def __init__(self):
        self.reset()
    
    def reset(self):
        # Dealer
        self.dealer_card =  self.get_card()
        self._dealer_usable_ace = self.dealer_card == '_A'
        # Player
        self.player_sum = 0
        self.usable_ace = False
        self.player_sum, self.usable_ace = self.hit(self.player_sum, self.usable_ace)
        self.player_sum, self.usable_ace = self.hit(self.player_sum, self.usable_ace)

    @staticmethod
    def hit(current_sum: int, usable_ace: bool):
        card = BlackJack.get_card()
        if card == '_A' and current_sum < 11:
            current_sum += 11
            usable_ace = True
        else:
            current_sum += BlackJack._CARDS[card]
            if current_sum > 21 and usable_ace:
                current_sum -= 10
                usable_ace = False
        return current_sum, usable_ace
    
    def sticks(self):
        dealer_sum = 11 if self._dealer_usable_ace else BlackJack._CARDS[self.dealer_card]
        while dealer_sum < 17:
            dealer_sum, self._dealer_usable_ace =  self.hit(dealer_sum, self._dealer_usable_ace)
        return dealer_sum, self._dealer_usable_ace

    @staticmethod
    def _rules(sum):
        if sum > 21:
            return -1 # Bust
        else:
            return 0 # Nothing happens

    def step(self, action):
        if action == 0:
            self.player_sum, self.usable_ace = self.hit(self.player_sum, self.usable_ace)
            if self._rules(self.player_sum) == -1:
                return True, -1, None
            else:
                return False, 0, (self.player_sum, self.usable_ace, self.dealer_card)
        elif action == 1:
            dealer_sum, _ = self.sticks()
            if self._rules(dealer_sum) == -1:
                return True, 1, None
            elif dealer_sum == self.player_sum:
                return True, 0, None
            elif dealer_sum < self.player_sum:
                return True, 1, None
            else:
                return True, 0, None
        else:
            raise Exception('Invalid action')

In [8]:
g = BlackJack()
g.step(0)

(True, -1, None)